In [1]:
import json
import requests
import pandas as pd

In [75]:
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
weights_API_list = ['TEN' , 'TWENTY' , 'FORTY' , 'TOTAL' , 'ETHEREUM' , 'CURRENCY' , 'PLATFORM' , 'APPLICATION']
old_month = 'july'
new_month = 'august'

In [68]:
indexes = dict.fromkeys(list_indexes)
#read in inputs and declare output variables
old_constants = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/All_'+old_month+'.csv' , index_col=[0])
new_values = old_constants * 1
new_values['Mkt Cap'] = old_constants['USD'] * 0
new_values['Adj Mkt Cap'] = old_constants['USD'] * 0
for k in indexes:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes[k] = pd.DataFrame()
        #indexes[k] = pd.read_csv('C:\Users\stoin\Desktop\Mike\Jupyter\Inputb\\'+k+'_'+old_month+'.csv' , index_col=[0])
        indexes[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+new_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes[k] = pd.DataFrame()
        #indexes[k] = pd.read_csv('C:\Users\stoin\Desktop\Mike\Jupyter\Inputb\\'+k+'_'+old_month+'.csv' , index_col=[0])
        indexes[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+new_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])

In [69]:
old_constants

,Divisor,Adj Divisor,USD,BTC,Adj USD,Adj BTC
10,2.467530e+08,8.095068e+07,837.908153,0.131971,2554.102607,0.402272
20,1.928513e+07,1.836063e+07,1074.882725,0.169294,1129.006032,0.177819
40,4.524594e+07,4.052111e+07,177.568846,0.027967,198.273675,0.031228
Total,6.447900e+08,6.562112e+08,365.248617,0.057527,358.891563,0.056526
ETH,8.330228e+07,8.437090e+07,125.361300,0.019744,123.773513,0.019494
Currency,3.190104e+09,0.000000e+00,43.918084,0.006917,0.000000,0.000000
Platform,2.261723e+09,0.000000e+00,37.864579,0.005964,0.000000,0.000000
Application,2.405478e+08,0.000000e+00,40.801156,0.006426,0.000000,0.000000


In [70]:
indexes['40']

,Float,Adj Float
40,,
RDD,2.880871e+10,4.904716e+10
DGB,1.045112e+10,4.896843e+09
ELF,2.500000e+08,3.152980e+08
BTG,1.709194e+07,7.527364e+06
AE,2.330205e+08,1.186396e+08
BTS,2.637120e+09,1.042222e+09
NAS,4.550000e+07,6.118949e+07
BCN,1.838900e+11,6.730265e+10
LRC,5.720740e+08,7.956458e+08


In [71]:
#API Call function
def daily_price_historical(symbol, base):
    url = 'https://min-api.cryptocompare.com/data/price?fsym={}&tsyms={}'\
            .format(symbol.upper(), base.upper())
    #print url
    #print t
    page = requests.get(url)
    df = page.json()['USD']
    #print df
    return df

In [72]:
#get BTC price
bitcoin = daily_price_historical('BTC','USD')

In [73]:
#calculate new index values, only do one for sectors (just one weighting scheme)
for k in indexes:
    #print indexes[k]
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        for t in indexes[k].index:
            try:
                df = daily_price_historical(t,'USD')
                indexes[k].loc[t,('Price')] = df
            except:
                print t , 'no price!'
        indexes[k]['Mkt Cap'] = indexes[k]['Price']*indexes[k]['Adj Float']
        indexes[k]['Weight'] = indexes[k]['Mkt Cap'] / indexes[k]['Mkt Cap'].sum()
        sum_weight = indexes[k]['Weight'].sum()
        print k, sum_weight
        new_values.loc[k,('Mkt Cap')] = indexes[k]['Mkt Cap'].sum()
        new_values.loc[k,('USD')] = indexes[k]['Mkt Cap'].sum() / new_values.loc[k,('Divisor')]
        new_values.loc[k,('BTC')] = new_values.loc[k,('USD')] / bitcoin
        if (indexes[k]['Price'] < 0.001).any() == True:
            print k, 'missing ticker!'
    else:
        for t in indexes[k].index:
            try:
                df = daily_price_historical(t,'USD')
                indexes[k].loc[t,('Price')] = df
            except:
                print t , 'no price!'
        indexes[k]['Mkt Cap'] = indexes[k]['Price']*indexes[k]['Float']
        indexes[k]['Weight'] = indexes[k]['Mkt Cap'] / indexes[k]['Mkt Cap'].sum()
        indexes[k]['Equal'] = 1.00 / (len(indexes[k]))
        indexes[k]['Adj Mkt Cap'] = indexes[k]['Price'] * indexes[k]['Adj Float']
        indexes[k]['Adj Weight'] = indexes[k]['Adj Mkt Cap'] / indexes[k]['Adj Mkt Cap'].sum()
        sum_weight = indexes[k]['Weight'].sum()
        sum_equal_weight = indexes[k]['Adj Weight'].sum()
        print k, sum_weight, sum_equal_weight
        new_values.loc[k,('Mkt Cap')] = indexes[k]['Mkt Cap'].sum()
        new_values.loc[k,('Adj Mkt Cap')] = indexes[k]['Adj Mkt Cap'].sum()
        new_values.loc[k,('USD')] = indexes[k]['Mkt Cap'].sum() / new_values.loc[k,('Divisor')]
        new_values.loc[k,('Adj USD')] = indexes[k]['Adj Mkt Cap'].sum() / new_values.loc[k,('Adj Divisor')]
        new_values.loc[k,('BTC')] = new_values.loc[k,('USD')] / bitcoin
        new_values.loc[k,('Adj BTC')] = new_values.loc[k,('Adj USD')] / bitcoin
        if (indexes[k]['Price'] < 0.001).any() == True:
            print k, 'missing ticker!'

10 1.0 1.0
20 1.0 1.0
Platform 1.0
40 1.0 1.0
Currency 1.0
Application 1.0
ETH 1.0 1.0
Total 1.0 1.0


In [74]:
new_values

,Divisor,Adj Divisor,USD,BTC,Adj USD,Adj BTC,Mkt Cap,Adj Mkt Cap
10,2.467530e+08,8.095068e+07,1022.692001,0.121482,3011.086436,0.357675,2.523523e+11,2.437495e+11
20,1.928513e+07,1.836063e+07,1095.679036,0.130151,1147.568082,0.136315,2.113032e+10,2.107007e+10
40,4.524594e+07,4.052111e+07,192.417663,0.022857,213.976669,0.025417,8.706118e+09,8.670572e+09
Total,6.447900e+08,6.562112e+08,437.471857,0.051966,405.281316,0.048142,2.820775e+11,2.659501e+11
ETH,8.330228e+07,8.437090e+07,122.967964,0.014607,117.888510,0.014004,1.024351e+10,9.946360e+09
Currency,3.190104e+09,0.000000e+00,56.858072,0.006754,0.000000,0.000000,1.813831e+11,0.000000e+00
Platform,2.261723e+09,0.000000e+00,40.337458,0.004792,0.000000,0.000000,9.123217e+10,0.000000e+00
Application,2.405478e+08,0.000000e+00,40.304345,0.004788,0.000000,0.000000,9.695123e+09,0.000000e+00


In [78]:
#write data to csv
new_values.to_csv('C:\Users\stoin\Desktop\Mike\Jupyter\Inputb\\All_pre_'+new_month+'.csv')

In [94]:
#read in input and declare output variables
indexes , indexes_old = dict.fromkeys(list_indexes) , dict.fromkeys(list_indexes)
indexes_clean = dict.fromkeys(list_indexes)
indexes_new , old_weights = dict.fromkeys(list_indexes) , dict.fromkeys(list_indexes)
new_weights , new_weights_clean = dict.fromkeys(list_indexes) , dict.fromkeys(list_indexes)
old_weights_clean , turnover = dict.fromkeys(list_indexes) , dict.fromkeys(list_indexes)
indexes_float_change = dict.fromkeys(list_indexes)
old_constants = new_values
new_constants = old_constants * 0
new_constants = new_constants.drop(['Mkt Cap' , 'Adj Mkt Cap'],axis=1)

fields = [0 , 4]
fields_sector = [0, 12]

for k in indexes:
    indexes_old[k] = pd.read_csv('C:\Users\stoin\Desktop\Mike\Jupyter\Inputb\\'+k+'_'+old_month+'.csv' , index_col=[0])
    indexes_float_change[k] = pd.DataFrame()
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes[k] = pd.DataFrame()
        indexes[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv' , index_col=[0])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes[k] = pd.DataFrame()
        indexes[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv' , index_col=[0])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float','Adj Float'])
    indexes_new[k] = indexes[k]

In [97]:
indexes['Currency']

,Float
ZEN,4157175
ZEC,4235169
XZC,5062809
XVG,15054315539
XMR,16173924
XLM,18760304485
PIVX,56594206
NANO,133248289
MONA,60474950
LTC,57168921


In [96]:
indexes_old['Currency']

,Adj Float
Currency,
ZEN,1.556484e+07
ZEC,1.585686e+07
XZC,1.895562e+07
XVG,5.636473e+10
XMR,6.055665e+07
XLM,7.024029e+10
PIVX,2.118939e+08
NANO,4.988938e+08
MONA,2.264237e+08


In [98]:
for k in indexes:
    print k , 'adds' , indexes_new[k].index.difference(indexes_old[k].index).values
    print k , 'deletes' , indexes_old[k].index.difference(indexes_new[k].index).values
    print k , 'count members:' , len(indexes_new[k])
    if indexes_new[k].index.duplicated(keep='first').any() == True:
        print 'Duplicates if True' , indexes_new[k].index.duplicated(keep='first')
    print ""
sectors = len(indexes_new['Currency'])+len(indexes_new['Platform'])+len(indexes_new['Application'])
print 'Sector count members:' , sectors

10 adds []
10 deletes []
10 count members: 10

20 adds []
20 deletes []
20 count members: 20

Platform adds []
Platform deletes []
Platform count members: 24

40 adds []
40 deletes []
40 count members: 40

Currency adds []
Currency deletes []
Currency count members: 19

Application adds []
Application deletes []
Application count members: 27

ETH adds []
ETH deletes []
ETH count members: 24

Total adds []
Total deletes []
Total count members: 70

Sector count members: 70


In [99]:
old_constants

,Divisor,Adj Divisor,USD,BTC,Adj USD,Adj BTC,Mkt Cap,Adj Mkt Cap
10,2.467530e+08,8.095068e+07,1022.692001,0.121482,3011.086436,0.357675,2.523523e+11,2.437495e+11
20,1.928513e+07,1.836063e+07,1095.679036,0.130151,1147.568082,0.136315,2.113032e+10,2.107007e+10
40,4.524594e+07,4.052111e+07,192.417663,0.022857,213.976669,0.025417,8.706118e+09,8.670572e+09
Total,6.447900e+08,6.562112e+08,437.471857,0.051966,405.281316,0.048142,2.820775e+11,2.659501e+11
ETH,8.330228e+07,8.437090e+07,122.967964,0.014607,117.888510,0.014004,1.024351e+10,9.946360e+09
Currency,3.190104e+09,0.000000e+00,56.858072,0.006754,0.000000,0.000000,1.813831e+11,0.000000e+00
Platform,2.261723e+09,0.000000e+00,40.337458,0.004792,0.000000,0.000000,9.123217e+10,0.000000e+00
Application,2.405478e+08,0.000000e+00,40.304345,0.004788,0.000000,0.000000,9.695123e+09,0.000000e+00


In [100]:
new_constants

,Divisor,Adj Divisor,USD,BTC,Adj USD,Adj BTC
10,0.0,0.0,0.0,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0
40,0.0,0.0,0.0,0.0,0.0,0.0
Total,0.0,0.0,0.0,0.0,0.0,0.0
ETH,0.0,0.0,0.0,0.0,0.0,0.0
Currency,0.0,0.0,0.0,0.0,0.0,0.0
Platform,0.0,0.0,0.0,0.0,0.0,0.0
Application,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
#main function to calculate new floats and divisors
for k in indexes:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):    #sectors are seperate due to capping
        for t in indexes[k].index:
            #print t
            try:
                df = daily_price_historical(t,'USD')
                indexes[k].loc[t,('Price')] = df
            except:
                print t
        if (k == 'Platform' or k == 'Application'):
            weight_cap = 0.2500
        else:
            weight_cap = 2.0000
        indexes[k]['Mkt Cap'] = indexes[k]['Price']*indexes[k]['Float']
        indexes[k]['Weight'] = indexes[k]['Mkt Cap'] / indexes[k]['Mkt Cap'].sum()
        indexes[k]['Weight Cap'] = weight_cap
        indexes[k]['Weight1'] = indexes[k]['Weight'] * 1
        #start looping through to handle the 25% capping rate
        for t in indexes[k].index:
            indexes[k].loc[t,('Weight1')] = min(indexes[k].loc[t,('Weight')] , indexes[k].loc[t,('Weight Cap')])
        sum1 = indexes[k]['Weight1'].sum()
        resid1 = 1.000 - sum1
        count1 = (sum(f > weight_cap for f in indexes[k]['Weight1'])) * weight_cap
        to_up1 = sum1 - count1
        mult1 = 1.000 + (resid1/to_up1)
        #print mult1 , to_up1, count1, resid1, sum1
        for t in indexes[k].index:
            if indexes[k].loc[t,('Weight1')] < (weight_cap - 0.0001):
                indexes[k].loc[t,('Weight_step2')] = indexes[k].loc[t,('Weight1')] * mult1
                indexes[k].loc[t,('Weight2')] = min(indexes[k].loc[t,('Weight_step2')] , indexes[k].loc[t,('Weight Cap')])
            else:
                indexes[k].loc[t,('Weight_step2')] = indexes[k].loc[t,('Weight1')]
                indexes[k].loc[t,('Weight2')] = indexes[k].loc[t,('Weight1')]
        sum2 = indexes[k]['Weight2'].sum()
        resid2 = 1.000 - sum2
        count2 = (sum(f >= weight_cap for f in indexes[k]['Weight2'])) * weight_cap
        to_up2 = sum2 - count2
        mult2 = 1.000 + (resid2/to_up2)
        #print mult2 , to_up2, count2, resid2, sum2
        for t in indexes[k].index:
            if indexes[k].loc[t,('Weight2')] < (weight_cap - 0.0001):
                indexes[k].loc[t,('Weight_step3')] = indexes[k].loc[t,('Weight2')] * mult2
                indexes[k].loc[t,('Weight3')] = min(indexes[k].loc[t,('Weight_step3')] , indexes[k].loc[t,('Weight Cap')])
            else:
                indexes[k].loc[t,('Weight_step3')] = indexes[k].loc[t,('Weight2')]
                indexes[k].loc[t,('Weight3')] = indexes[k].loc[t,('Weight2')]
        sum3 = indexes[k]['Weight3'].sum()
        resid3 = 1.000 - sum3
        count3 = (sum(f >= weight_cap for f in indexes[k]['Weight3'])) * weight_cap
        to_up3 = sum3 - count3
        mult3 = 1.000 + (resid3/to_up3)
        #print mult3 , to_up3, count3, resid3, sum3
        for t in indexes[k].index:
            if indexes[k].loc[t,('Weight3')] <= weight_cap:
                indexes[k].loc[t,('Weight_step4')] = indexes[k].loc[t,('Weight3')] * mult3
                indexes[k].loc[t,('Weight4')] = min(indexes[k].loc[t,('Weight_step4')] , indexes[k].loc[t,('Weight Cap')])
            else:
                indexes[k].loc[t,('Weight_step4')] = indexes[k].loc[t,('Weight3')]
                indexes[k].loc[t,('Weight4')] = indexes[k].loc[t,('Weight3')]
        #print indexes[k]
        sum4 = indexes[k]['Weight4'].sum()
        resid4 = 1.000 - sum4
        count4 = (sum(f >= weight_cap for f in indexes[k]['Weight4'])) * weight_cap
        to_up4 = sum4 - count4
        mult4 = 1.000 + (resid4/to_up4)
        print k, mult4 , to_up4, count4, resid4, sum4
        #done iterating to compute proper adjusted weights
        indexes[k]['AWF'] = indexes[k]['Weight4'] / indexes[k]['Weight']
        indexes[k]['Adj Mkt Cap'] = indexes[k]['Mkt Cap'] * indexes[k]['AWF']
        indexes[k]['Adj Weight'] = indexes[k]['Adj Mkt Cap'] / indexes[k]['Adj Mkt Cap'].sum()
        indexes[k]['Adj Float'] = indexes[k]['Float'] * indexes[k]['AWF']
        sum_equal_weight = indexes[k]['Adj Weight'].sum()
        new_constants.loc[k,('Divisor')] = old_constants.loc[k,('Divisor')] *\
        (indexes[k]['Adj Mkt Cap'].sum() / old_constants.loc[k,('Mkt Cap')])
        new_constants.loc[k,('USD')] = indexes[k]['Adj Mkt Cap'].sum() / new_constants.loc[k,('Divisor')]
        new_constants.loc[k,('BTC')] = new_constants.loc[k,('USD')] / bitcoin
    else:
        for t in indexes[k].index:
            #print t
            try:
                df = daily_price_historical(t,'USD')
                indexes[k].loc[t,('Price')] = df
            except:
                print t
        indexes[k]['Mkt Cap'] = indexes[k]['Price']*indexes[k]['Float']
        indexes[k]['Weight'] = indexes[k]['Mkt Cap'] / indexes[k]['Mkt Cap'].sum()
        indexes[k]['Equal'] = 1.00 / (len(indexes[k]))
        indexes[k]['AWF'] = indexes[k]['Equal'] / indexes[k]['Weight']
        indexes[k]['Adj Mkt Cap'] = indexes[k]['Mkt Cap'] * indexes[k]['AWF']
        indexes[k]['Adj Weight'] = indexes[k]['Adj Mkt Cap'] / indexes[k]['Adj Mkt Cap'].sum()
        indexes[k]['Adj Float'] = indexes[k]['Float'] * indexes[k]['AWF']
        sum_weight = indexes[k]['Weight'].sum()
        sum_equal_weight = indexes[k]['Adj Weight'].sum()
        print k, sum_weight, sum_equal_weight, indexes[k]['Weight'].max(), indexes[k]['Weight'].min()
        new_constants.loc[k,('Divisor')] = old_constants.loc[k,('Divisor')] *\
        (indexes[k]['Mkt Cap'].sum() / old_constants.loc[k,('Mkt Cap')])
        new_constants.loc[k,('Adj Divisor')] = old_constants.loc[k,('Adj Divisor')] *\
        (indexes[k]['Adj Mkt Cap'].sum() / old_constants.loc[k,('Adj Mkt Cap')])
        new_constants.loc[k,('USD')] = indexes[k]['Mkt Cap'].sum() / new_constants.loc[k,('Divisor')]
        new_constants.loc[k,('Adj USD')] = indexes[k]['Adj Mkt Cap'].sum() / new_constants.loc[k,('Adj Divisor')]
        new_constants.loc[k,('BTC')] = new_constants.loc[k,('USD')] / bitcoin
        new_constants.loc[k,('Adj BTC')] = new_constants.loc[k,('Adj USD')] / bitcoin

10 1.0 1.0 0.569928327749 0.0068215996444
20 1.0 1.0 0.129721328191 0.0103125879222
Platform 1.0 0.5 0.5 0.0 1.0
40 1.0 1.0 0.0656860260977 0.00666609238678
Currency 1.0 1.0 0.0 0.0 1.0
Application 1.0 0.75 0.25 0.0 1.0
ETH 1.0 1.0 0.247584991834 0.00563356270941
Total 1.0 1.0 0.509441292472 0.000205439381296


In [102]:
indexes['Currency']

,Float,Price,Mkt Cap,Weight,Weight Cap,Weight1,Weight_step2,Weight2,Weight_step3,Weight3,Weight_step4,Weight4,AWF,Adj Mkt Cap,Adj Weight,Adj Float
ZEN,4157175,32.38000,1.346093e+08,0.000742,2.0,0.000742,0.000742,0.000742,0.000742,0.000742,0.000742,0.000742,1.0,1.346093e+08,0.000742,4.157175e+06
ZEC,4235169,221.02000,9.360571e+08,0.005162,2.0,0.005162,0.005162,0.005162,0.005162,0.005162,0.005162,0.005162,1.0,9.360571e+08,0.005162,4.235169e+06
XZC,5062809,16.43000,8.318195e+07,0.000459,2.0,0.000459,0.000459,0.000459,0.000459,0.000459,0.000459,0.000459,1.0,8.318195e+07,0.000459,5.062809e+06
XVG,15054315539,0.02403,3.617552e+08,0.001995,2.0,0.001995,0.001995,0.001995,0.001995,0.001995,0.001995,0.001995,1.0,3.617552e+08,0.001995,1.505432e+10
XMR,16173924,148.43000,2.400696e+09,0.013239,2.0,0.013239,0.013239,0.013239,0.013239,0.013239,0.013239,0.013239,1.0,2.400696e+09,0.013239,1.617392e+07
XLM,18760304485,0.30220,5.669364e+09,0.031264,2.0,0.031264,0.031264,0.031264,0.031264,0.031264,0.031264,0.031264,1.0,5.669364e+09,0.031264,1.876030e+10
PIVX,56594206,1.83000,1.035674e+08,0.000571,2.0,0.000571,0.000571,0.000571,0.000571,0.000571,0.000571,0.000571,1.0,1.035674e+08,0.000571,5.659421e+07
NANO,133248289,2.34000,3.118010e+08,0.001719,2.0,0.001719,0.001719,0.001719,0.001719,0.001719,0.001719,0.001719,1.0,3.118010e+08,0.001719,1.332483e+08
MONA,60474950,2.24000,1.354639e+08,0.000747,2.0,0.000747,0.000747,0.000747,0.000747,0.000747,0.000747,0.000747,1.0,1.354639e+08,0.000747,6.047495e+07
LTC,57168921,89.09000,5.093179e+09,0.028087,2.0,0.028087,0.028087,0.028087,0.028087,0.028087,0.028087,0.028087,1.0,5.093179e+09,0.028087,5.716892e+07


In [103]:
#check for ticker issues
for k in indexes:
    #print indexes[k]['Price']
    if (indexes[k]['Price'] < 0.0001).any() == True:
            print k, 'missing ticker!' , indexes[k]['Price']

In [104]:
new_constants

,Divisor,Adj Divisor,USD,BTC,Adj USD,Adj BTC
10,2.471299e+08,8.393575e+07,1022.692001,0.121482,3011.086436,0.357675
20,1.933682e+07,1.846248e+07,1095.679036,0.130151,1147.568082,0.136315
40,4.526267e+07,4.070228e+07,192.417663,0.022857,213.976669,0.025417
Total,6.457395e+08,6.970291e+08,437.471857,0.051966,405.281316,0.048142
ETH,8.379126e+07,8.740157e+07,122.967964,0.014607,117.888510,0.014004
Currency,3.189281e+09,0.000000e+00,56.858072,0.006754,0.000000,0.000000
Platform,2.267506e+09,0.000000e+00,40.337458,0.004792,0.000000,0.000000
Application,2.414613e+08,0.000000e+00,40.304345,0.004788,0.000000,0.000000


In [105]:
#total market cap
mkt_cap = new_constants.loc['Total']['Divisor'] * new_constants.loc['Total']['USD']
print mkt_cap*0.000000001

282.492861076


In [106]:
for k in indexes:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_float_change[k]['Float Change'] = indexes[k]['Adj Float'] - indexes_old[k]['Adj Float']
        indexes_float_change[k]['Float Change Percent'] = (indexes[k]['Adj Float'] / indexes_old[k]['Adj Float']) - 1
    else:
        indexes_float_change[k]['Float Change'] = indexes_new[k]['Float'] - indexes_old[k]['Float']
        indexes_float_change[k]['Float Change Percent'] = (indexes_new[k]['Float'] / indexes_old[k]['Float']) - 1
indexes_float_change

{'10':       Float Change  Float Change Percent
 BTC              0                   0.0
 ETH              0                   0.0
 XRP              0                   0.0
 XLM              0                   0.0
 BCH              0                   0.0
 EOS              0                   0.0
 LTC              0                   0.0
 ADA              0                   0.0
 ETC              0                   0.0
 DASH             0                   0.0,
 '20':        Float Change  Float Change Percent
 ONT             0.0          0.000000e+00
 TRX             0.0          0.000000e+00
 ZEC             0.0          0.000000e+00
 NEO             0.0          0.000000e+00
 XMR             0.0          0.000000e+00
 IOT             0.0          0.000000e+00
 VEN             0.0          0.000000e+00
 DCR             0.0          0.000000e+00
 BNB             0.0          0.000000e+00
 XEM             0.0          0.000000e+00
 ZIL             0.0          0.000000e+00
 MAID    

In [107]:
with open('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/java/com/frobro/bcindex/web/constants/Test.java','r') as constants:
    lines = constants.readlines()

In [108]:
ten_divisor = new_constants.loc['10']['Divisor'].astype(str)
ten_divisor_even = new_constants.loc['10']['Adj Divisor'].astype(str)
twenty_divisor = new_constants.loc['20']['Divisor'].astype(str)
twenty_divisor_even = new_constants.loc['20']['Adj Divisor'].astype(str)
forty_divisor = new_constants.loc['40']['Divisor'].astype(str)
forty_divisor_even = new_constants.loc['40']['Adj Divisor'].astype(str)
total_divisor = new_constants.loc['Total']['Divisor'].astype(str)
total_divisor_even = new_constants.loc['Total']['Adj Divisor'].astype(str)
eth_divisor = new_constants.loc['ETH']['Divisor'].astype(str)
eth_divisor_even = new_constants.loc['ETH']['Adj Divisor'].astype(str)
currency_divisor = new_constants.loc['Currency']['Divisor'].astype(str)
platform_divisor = new_constants.loc['Platform']['Divisor'].astype(str)
application_divisor = new_constants.loc['Application']['Divisor'].astype(str)

In [109]:
new_csv_file_lines = {8 , 9 , 12 , 18 , 24 , 30 , 36 , 41 , 46 , 51 , 16 , 17,22,23,28,29,34,35,40,45,50}
indexes_csv = {'10' , '20' , '40' , 'ETH' , 'Total' , 'Currency' , 'Platform' , 'Application'}
for l in new_csv_file_lines:
    #print l
    if l == 12:
        lines[l] = '  public static final String MKT_CAP_FILE = "business_rules/10_'+new_month+'_rebal.csv";\n'
    elif l == 8:
        lines[l] = '  public static final double DIVISOR_TEN = '+ten_divisor+';\n'
    elif l == 9:
        lines[l] = '  public static final double DIVISOR_EVEN_TEN = '+ten_divisor_even+';\n'
    elif l == 16:
        lines[l] = '  public static final double DIVISOR_20 = '+twenty_divisor+';\n'
    elif l == 17:
        lines[l] = '  public static final double DIVISOR_EVEN_20 = '+twenty_divisor_even+';\n'
    elif l == 22:
        lines[l] = '  public static final double DIVISOR_40 = '+forty_divisor+';\n'
    elif l == 23:
        lines[l] = '  public static final double DIVISOR_EVEN_40 = '+forty_divisor_even+';\n'
    elif l == 34:
        lines[l] = '  public static final double DIVISOR_TOTAL = '+total_divisor+';\n'
    elif l == 35:
        lines[l] = '  public static final double DIVISOR_EVEN_TOTAL = '+total_divisor_even+';\n'
    elif l == 28:
        lines[l] = '  public static final double DIVISOR_ETHER = '+eth_divisor+';\n'
    elif l == 29:
        lines[l] = '  public static final double DIVISOR_EVEN_ETHER = '+eth_divisor_even+';\n'
    elif l == 40:
        lines[l] = '  public static final double DIVISOR_CURRENCY = '+currency_divisor+';\n'
    elif l == 45:
        lines[l] = '  public static final double DIVISOR_PLATFORM = '+platform_divisor+';\n'
    elif l == 50:
        lines[l] = '  public static final double DIVISOR_APPLICATION = '+application_divisor+';\n'
    elif l == 18:
        lines[l] = '  public static final String MKT_CAP_FILE_20 = "business_rules/20_'+new_month+'_rebal.csv";\n'
    elif l == 24:
        lines[l] = '  public static final String MKT_CAP_FILE_40 = "business_rules/40_'+new_month+'_rebal.csv";\n'
    elif l == 30:
        lines[l] = '  public static final String MKT_CAP_FILE_ETHER = "business_rules/ETH_'+new_month+'_rebal.csv";\n'
    elif l == 36:
        lines[l] = '  public static final String MKT_CAP_FILE_TOTAL = "business_rules/Total_'+new_month+'_rebal.csv";\n'
    elif l == 41:
        lines[l] = '  public static final String MKT_CAP_FILE_CURRENCY = "business_rules/Currency_'+new_month+'_rebal.csv";\n'
    elif l == 46:
        lines[l] = '  public static final String MKT_CAP_FILE_PLATFORM = "business_rules/Platform_'+new_month+'_rebal.csv";\n'
    elif l == 51:
        lines[l] = '  public static final String MKT_CAP_FILE_APPLICATION = "business_rules/Application_'+new_month+'_rebal.csv";\n'

In [110]:
with open('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/java/com/frobro/bcindex/web/constants/Test.java','w') as constants:
    constants.writelines( lines )
constants.close()

In [111]:
#write data to csv
new_constants.to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/All_'+new_month+'.csv')
indexes_final = dict.fromkeys(list_indexes)
for k in indexes:
    #indexes[k].to_csv('C:\Users\stoin\Desktop\Mike\Jupyter\Inputb\\'+k+'_'+new_month+'_weights.csv')
    #indexes_float_change[k].to_csv('C:\Users\stoin\Desktop\Mike\Jupyter\Inputb\\'+k+'_'+new_month+'_float_change.csv')
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_final[k] = indexes[k].drop(['Float' , 'Price' , 'Mkt Cap' , 'Weight' , 'Weight Cap' , 'Weight1' , 'Weight_step2' , 'Weight2' \
        , 'Weight_step3' , 'Weight3' , 'Weight_step4' , 'Weight4' , 'AWF' , 'Adj Mkt Cap' , 'Adj Weight'],axis=1)
    else:
        indexes_final[k] = indexes[k].drop(['Price' , 'Mkt Cap' , 'Weight' , 'Equal' , 'AWF' , 'Adj Mkt Cap' , 'Adj Weight'],axis=1)
    #indexes_final[k].to_csv('C:\Users\stoin\Desktop\Mike\Jupyter\Inputb\\'+k+'_'+new_month+'.csv')
    #indexes_final[k].to_csv('C:\Users\stoin\Desktop\Mike\Jupyter\Inputb\\'+k+'_'+new_month+'_rebal.csv' , header = False)
    indexes_final[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+new_month+'_rebal.csv' , header = False)

In [112]:
weights_API_list = ['TEN' , 'TWENTY' , 'FORTY' , 'TOTAL' , 'ETHEREUM' , 'CURRENCY' , 'PLATFORM' , 'APPLICATION']
endPoint = "https://www.bletchleyindexes.com/api/weight";
for k in weights_API_list:
    #print (k)
    data = {'index':k}
    #print (data)
    old_weights[k] = pd.DataFrame(columns=['Weight'])
    #print old_weights[k]
    response = requests.post(endPoint, json=data).text
    #print(response)
    old_weights[k]['Weight'] = pd.read_json(response, typ='series' , encoding='utf-8')
    #print (old_weights[k])
#old_weights[k]

In [113]:
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
weights_API_list = ['TEN' , 'TWENTY' , 'FORTY' , 'TOTAL' , 'ETHEREUM' , 'CURRENCY' , 'PLATFORM' , 'APPLICATION']
weights_master = dict.fromkeys(list_indexes)
old_weights['10'] = old_weights['TEN']
old_weights['20'] = old_weights['TWENTY']
old_weights['40'] = old_weights['FORTY']
old_weights['ETH'] = old_weights['ETHEREUM']
old_weights['Currency'] = old_weights['CURRENCY']
old_weights['Platform'] = old_weights['PLATFORM']
old_weights['Application'] = old_weights['APPLICATION']
old_weights['Total'] = old_weights['TOTAL']
print old_weights['10']

        Weight
ADA   0.018091
BCH   0.059533
BTC   0.569308
DASH  0.008162
EOS   0.031658
ETC   0.006866
ETH   0.191638
LTC   0.020180
XLM   0.022542
XRP   0.072023


In [115]:
indexes['10']

,Float,Price,Mkt Cap,Weight,Equal,AWF,Adj Mkt Cap,Adj Weight,Adj Float
BTC,17119662,8413.8600,1.440424e+11,0.569928,0.1,0.175461,2.527378e+10,0.1,3.003827e+06
ETH,100352655,482.0500,4.837500e+10,0.191404,0.1,0.522455,2.527378e+10,0.1,5.242979e+07
XRP,39262097329,0.4623,1.815087e+10,0.071817,0.1,1.392428,2.527378e+10,0.1,5.466965e+10
XLM,18760304485,0.3022,5.669364e+09,0.022432,0.1,4.457957,2.527378e+10,0.1,8.363263e+10
BCH,17207875,875.9200,1.507272e+10,0.059638,0.1,1.676789,2.527378e+10,0.1,2.885398e+07
EOS,896149492,8.9000,7.975730e+09,0.031557,0.1,3.168836,2.527378e+10,0.1,2.839751e+09
LTC,57168921,89.0900,5.093179e+09,0.020152,0.1,4.962280,2.527378e+10,0.1,2.836882e+08
ADA,25927070538,0.1760,4.563164e+09,0.018055,0.1,5.538652,2.527378e+10,0.1,1.436010e+11
ETC,102623582,16.8000,1.724076e+09,0.006822,0.1,14.659318,2.527378e+10,0.1,1.504392e+09
DASH,8158151,253.8900,2.071273e+09,0.008195,0.1,12.202052,2.527378e+10,0.1,9.954619e+07


In [116]:
indexes['Currency']

,Float,Price,Mkt Cap,Weight,Weight Cap,Weight1,Weight_step2,Weight2,Weight_step3,Weight3,Weight_step4,Weight4,AWF,Adj Mkt Cap,Adj Weight,Adj Float
ZEN,4157175,32.38000,1.346093e+08,0.000742,2.0,0.000742,0.000742,0.000742,0.000742,0.000742,0.000742,0.000742,1.0,1.346093e+08,0.000742,4.157175e+06
ZEC,4235169,221.02000,9.360571e+08,0.005162,2.0,0.005162,0.005162,0.005162,0.005162,0.005162,0.005162,0.005162,1.0,9.360571e+08,0.005162,4.235169e+06
XZC,5062809,16.43000,8.318195e+07,0.000459,2.0,0.000459,0.000459,0.000459,0.000459,0.000459,0.000459,0.000459,1.0,8.318195e+07,0.000459,5.062809e+06
XVG,15054315539,0.02403,3.617552e+08,0.001995,2.0,0.001995,0.001995,0.001995,0.001995,0.001995,0.001995,0.001995,1.0,3.617552e+08,0.001995,1.505432e+10
XMR,16173924,148.43000,2.400696e+09,0.013239,2.0,0.013239,0.013239,0.013239,0.013239,0.013239,0.013239,0.013239,1.0,2.400696e+09,0.013239,1.617392e+07
XLM,18760304485,0.30220,5.669364e+09,0.031264,2.0,0.031264,0.031264,0.031264,0.031264,0.031264,0.031264,0.031264,1.0,5.669364e+09,0.031264,1.876030e+10
PIVX,56594206,1.83000,1.035674e+08,0.000571,2.0,0.000571,0.000571,0.000571,0.000571,0.000571,0.000571,0.000571,1.0,1.035674e+08,0.000571,5.659421e+07
NANO,133248289,2.34000,3.118010e+08,0.001719,2.0,0.001719,0.001719,0.001719,0.001719,0.001719,0.001719,0.001719,1.0,3.118010e+08,0.001719,1.332483e+08
MONA,60474950,2.24000,1.354639e+08,0.000747,2.0,0.000747,0.000747,0.000747,0.000747,0.000747,0.000747,0.000747,1.0,1.354639e+08,0.000747,6.047495e+07
LTC,57168921,89.09000,5.093179e+09,0.028087,2.0,0.028087,0.028087,0.028087,0.028087,0.028087,0.028087,0.028087,1.0,5.093179e+09,0.028087,5.716892e+07


In [118]:
sectors = ['Application' , 'Platform' , 'Currency']
for k in indexes:
    new_weights[k] = pd.DataFrame()
    if k in sectors:
        #new_weights[k] = pd.read_csv('C:\Users\stoin\Desktop\Mike\Jupyter\Inputb\\'+k+'_'+new_month+'_weights.csv' , index_col=[0] , usecols=fields_sector)
        new_weights[k]['Weight'] = indexes[k]['Adj Weight']
    else:
        #print k
        #fields = [0,4]
        #new_weights[k] = pd.read_csv('C:\Users\stoin\Desktop\Mike\Jupyter\Inputb\\'+k+'_'+new_month+'_weights.csv' , index_col=[0] , usecols=fields)
        #print new_weights[k]
        new_weights[k]['Weight'] = indexes[k]['Weight']
turnover_summary = pd.DataFrame(index=list_indexes , columns=['Turnover'])
for k in indexes:
    print k
    #print new_weights[k]
    adds = indexes_new[k].index.difference(indexes_old[k].index).values
    deletes = indexes_old[k].index.difference(indexes_new[k].index).values
    turnover[k] = pd.DataFrame(columns=['Turnover'])
    #print k , adds
    for t in adds:
        new_ticker = {'Weight': [0]}
        new_add = pd.DataFrame(data = new_ticker)
        #print new_add
        #print t
        #print old_weights[k]
        old_weights[k] = old_weights[k].append([new_add])
        old_weights[k].index = old_weights[k].index[:-1].append(pd.Index([t]))
        #print old_weights[k]
    #print k , deletes
    for t in deletes:
        new_ticker = {'Weight': [0]}
        new_delete = pd.DataFrame(data = new_ticker)
        new_weights[k] = new_weights[k].append([new_delete])
        new_weights[k].index = new_weights[k].index[:-1].append(pd.Index([t]))
    new_weights[k] = new_weights[k].fillna(0)
    #print new_weights[k]
    new_weights_clean[k] = pd.DataFrame(columns=['Weight'])
    new_weights_clean[k]['Weight'] = new_weights[k]['Weight']
    #print new_weights_clean[k]
    old_weights[k] = old_weights[k].fillna(0)
    old_weights_clean[k] = old_weights[k]
    #print old_weights_clean[k]
    turnover[k]['Turnover'] = (new_weights_clean[k]['Weight'] - old_weights_clean[k]['Weight']).abs()
    #print turnover[k]
    weights_master[k] = pd.DataFrame()
    weights_master[k]['New Weight'] = new_weights_clean[k]['Weight']
    weights_master[k]['Old Weight'] = old_weights_clean[k]['Weight']
    weights_master[k]['Turnover'] = turnover[k]['Turnover']
    weights_master[k] = weights_master[k].fillna(0)
    #print turnover[k]
    turnover_summary.loc[k,('Turnover')] = turnover[k]['Turnover'].abs().sum()/2
    #print k , (turnover[k].abs().sum()*100)/2

10
20
Platform
40
Currency
Application
ETH
Total


In [121]:
#writer = pd.ExcelWriter('C:\Users\stoin\Desktop\Mike\Jupyter\Inputb\\new_weights_'+new_month+'.xlsx', engine='xlsxwriter')
upload_weights = pd.DataFrame()
upload_weights = pd.concat([turnover_summary ])
upload_floats = pd.DataFrame()
for k in indexes:
    index_name = pd.DataFrame(columns = ['New Weight' , 'Old Weight' , 'Turnover'])
    index_name.set_index = (k+' Index')
    index_name.loc[k] = ['New Weight' , 'Old Weight' , 'Turnover']
    upload_weights = upload_weights.append(index_name)
    upload_weights = pd.concat([upload_weights , weights_master[k]])
    index_float_name = pd.DataFrame(columns = ['Float Change' , 'Float Change Percent'])
    index_float_name.set_index = (k+' Index')
    index_float_name.loc[k] = ['Float Change' , 'Float Change Percent']
    upload_floats = upload_floats.append(index_float_name)
    upload_floats = pd.concat([upload_floats , indexes_float_change[k]])
upload_weights
#upload_weights.to_csv('C:\Users\stoin\Desktop\Mike\Jupyter\Inputb\\new_weights_'+new_month+'.csv')
upload_weights.to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-view/src/main/resources/static/weights/'+new_month+'_2018.csv')
#upload_floats.to_csv('C:\Users\stoin\Desktop\Mike\Jupyter\Inputb\\new_floats_'+new_month+'.csv')
upload_floats.to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-view/src/main/resources/static/weights/'+new_month+'_floats_2018.csv')
#turnover_summary.to_excel(writer , sheet_name='Turnover Summary')
#writer.save()

In [122]:
turnover_summary

,Turnover
10,0.000758182
20,0.00275285
40,0.00266394
Total,0.000969219
ETH,0.00292795
Currency,0.523852
Platform,0.0163445
Application,0.00809897
